# Data Storage and Access Patterns (including Astra Vector Search)



# Table Schema

```
CREATE TABLE IF NOT EXISTS workspan.customer_opportunities (
    customer_id text,
    partner_id text,
    opportunity_id text,
    customer_name text static,
    next_step text,
    cadence text,
    llm_output text,
    opportunity map<text, text>,
    llm_output_embedding vector<float, 1536>,
    sentiment text,
    PRIMARY KEY ((customer_id, partner_id), opportunity_id)
) WITH CLUSTERING ORDER BY (opportunity_id DESC)

```



* customer_id: Unique identifier for each customer
* partner_id: AWS / Azure / GCP
* opportunity_id: Unique identifier for each opportunity
* opportunity: Dynamic collection of data field names and corresponding values specific to the opportunity. Other fields can be stored in separate table columns.
* llm_ouput: LLM output summarizing the 'next steps' , 'challenges' and 'open items'  
* llm_ouput_embedding: Text embeddings corresponding to llm_output
* sentiment: Positive, Neutral and Negative sentiment derived from 'next step' and 'cadence' data fields. This can be determined either by using a sentiment analysis library such as  Python Natural Language Toolkit (NLTK) or from LLM.


# Imports

In [ ]:
!pip install openai cassandra-driver llama-index

In [2]:
from cassandra.cluster import Cluster
from cassandra.auth import PlainTextAuthProvider
from cassandra.query import dict_factory
from cassandra.query import SimpleStatement
import openai
from llama_index import ListIndex
from llama_index.readers.schema.base import Document
from IPython.display import Markdown, display

# Keys & Environment Variables

In [3]:
# keys and tokens here
openai_api_key = ""
openai.api_key = openai_api_key
cass_user = 'client-id'
cass_pw = 'client-secret'
scb_path = 'secure-connect-bundle.zip'

# Select a model to compute embeddings

In [4]:
model_id = "text-embedding-ada-002"

# Connect to the Cluster

In [6]:
cloud_config= {
  'secure_connect_bundle': scb_path
}
auth_provider = PlainTextAuthProvider(cass_user, cass_pw)
cluster = Cluster(cloud=cloud_config, auth_provider=auth_provider)
session = cluster.connect()
session.set_keyspace('workspan')

ERROR:cassandra.connection:Closing connection <AsyncoreConnection(137539020904656) b2d68f45-7d3e-4936-a50c-b0742b34b3f5-us-east-2.db.astra.datastax.com:29042:6d1f8fed-4e79-42bd-8cb2-7f2dc0b56183> due to protocol error: Error from server: code=000a [Protocol error] message="Beta version of the protocol used (5/v5-beta), but USE_BETA flag is unset"


# Drop / Create Schema

In [7]:

# # Create Table
session.execute(f"""CREATE TABLE IF NOT EXISTS workspan.customer_opportunities (
    customer_id text,
    partner_id text,
    opportunity_id text,
    customer_name text static,
    next_step text,
    cadence text,
    llm_output text,
    opportunity map<text, text>,
    llm_output_embedding vector<float, 1536>,
    sentiment text,
    PRIMARY KEY ((customer_id, partner_id), opportunity_id)
) WITH CLUSTERING ORDER BY (opportunity_id DESC)""")



In [ ]:
session.execute(f"""CREATE CUSTOM INDEX ON workspan.customer_opportunities(opportunity) USING 'org.apache.cassandra.index.sai.StorageAttachedIndex'""")

In [ ]:
session.execute(f"""CREATE CUSTOM INDEX ON workspan.customer_opportunities (llm_output_embedding) USING 'org.apache.cassandra.index.sai.StorageAttachedIndex'""")

In [ ]:
session.execute(f"""CREATE CUSTOM INDEX ON workspan.customer_opportunities (sentiment) USING 'org.apache.cassandra.index.sai.StorageAttachedIndex'""")

# Create the following SAI with analyser from CQL


```
CREATE CUSTOM INDEX ON workspan.customer_opportunities(llm_output) USING 'org.apache.cassandra.index.sai.StorageAttachedIndex' WITH OPTIONS = {'index_analyzer': '{
"tokenizer" : {"name" : "standard"},
"filters" : [{"name" : "porterstem"}] }'};
```



# Calculate the sentiment

Sentiment can be calculated either by using a sentiment analysis library such as Python Natural Language Toolkit (NLTK) or from LLM.

In [11]:
def indentify_sentiment(next_step_and_cadence):
    import nltk
    from nltk.sentiment.vader import SentimentIntensityAnalyzer

    nltk.download('vader_lexicon')
    sid = SentimentIntensityAnalyzer()

    sentiment_scores = sid.polarity_scores(next_step_and_cadence)
    if sentiment_scores['compound'] >= 0.05:
        sentiment = 'positive'
    elif sentiment_scores['compound'] <= -0.05:
        sentiment = 'negative'
    else:
        sentiment = 'neutral'

    print(f"Sentiment: {sentiment}")
    return sentiment


# Generate LLM ouput

In [14]:
def extract_llm_information(next_step_and_cadence):

    llm_input = f"""
                Given the following information, please identify the challenges, next steps, and open items. If there is a mention of any meetings that need to be scheduled, please list it under 'schedule a meeting:'. If there's no indication of a meeting, then avoid the listing:

                {next_step_and_cadence}

                End of information.
                """

    response = openai.chat.completions.create(
        #model="gpt-3.5-turbo",
        model="gpt-4",
        messages=[
            {"role": "system", "content": "You are a helpful assistant. Use the instructions provided to process the information."},
            {"role": "user", "content": llm_input}
        ]
    )

    return response.choices[0].message.content

# Insert records with the calculated sentiment and LLM embeddings

In [21]:
query = SimpleStatement(
            f"""
            INSERT INTO workspan.customer_opportunities
            (customer_id, partner_id, opportunity_id, customer_name, llm_output,  opportunity , llm_output_embedding , sentiment )
            VALUES (%s, %s, %s, %s, %s , %s , %s , %s )
            """
        )

# record #1

next_step = f"""
Action Items:
From Michael, confirmed deprioritize. From Anjaney, account executive interest to schedule meeting - Anjaney to schedule call with Nirav/Amy on R&D.
"""

cadence = f"""
Next Step:
08/16/2023 : Review partner information updates and update opportunity details. 8/17(LR) - connecting with Partner to offer co-sell support

Next Step History:
null;08/16/2023 : Review partner information updates and update opportunity details.;08/16/2023 : Review partner information updates and update opportunity details. 8/17(LR) - connecting with Partner to offer co-sell support
"""

next_step_and_cadence = next_step + cadence

sentiment = indentify_sentiment(next_step_and_cadence)
llm_output = extract_llm_information(next_step_and_cadence)
print(f"{llm_output}/n")
embedding_llm_output = openai.embeddings.create(input= llm_output, model=model_id).data[0].embedding
session.execute(query, ('CUS100' , 'AWS', 'WS-7202838a', 'Teradyne, Inc.', llm_output,  {'Customer State' : 'Ile-de-France', 'Customer Country' : 'France', 'Deal Size' : '30000', 'Description' : 'Pain Point: Persistent phishing attacks and email compromises leading to data leaks and compromised accounts. Description: The customer is struggling with the recurring threat of phishing attacks that infiltrate their systems, compromising sensitive data and risking the confidentiality of critical information.' }
                        , embedding_llm_output , sentiment))

# record #2

next_step = f"""
Action Items:
From Autumn, send recording of last call and our discussed inputs from demo 8/28. Ramesh will provide to Caroline by early next week (of 9/11).
"""

cadence = f"""
REVIEW TECH & Economic Proposal
"""

next_step_and_cadence = next_step + cadence

sentiment = indentify_sentiment(next_step_and_cadence)
llm_output = extract_llm_information(next_step_and_cadence)
print(f"{llm_output}/n")
embedding_llm_output = openai.embeddings.create(input= llm_output, model=model_id).data[0].embedding
session.execute(query, ('CUS100' , 'AWS', 'WS-8a038b8a', 'Teradyne, Inc.', llm_output,  {'Customer State' : 'Ile-de-France', 'Customer Country' : 'France', 'Deal Size' : '30000', 'Description' : 'Pain Point: Persistent phishing attacks and email compromises leading to data leaks and compromised accounts. Description: The customer is struggling with the recurring threat of phishing attacks that infiltrate their systems, compromising sensitive data and risking the confidentiality of critical information.' }
                        , embedding_llm_output , sentiment))

# record #3

next_step = f"""
Action Items:
Joint sync set for 9/7. Enablement session to follow + in person account mapping. Caroline / Michael to begin coordinating. EAI presence
"""

cadence = f"""
07/05/2023: Contact Federico Gandolfo,federico.hernan.gandolfo@abc.com,+54.911.3204.4871 to discuss Deal support
"""

next_step_and_cadence = next_step + cadence

sentiment = indentify_sentiment(next_step_and_cadence)
llm_output = extract_llm_information(next_step_and_cadence)
print(f"{llm_output}/n")
embedding_llm_output = openai.embeddings.create(input= llm_output, model=model_id).data[0].embedding
session.execute(query, ('CUS100' , 'AWS', 'WS-8a3b0348', 'Teradyne, Inc.', llm_output,  {'Customer State' : 'Ile-de-France', 'Customer Country' : 'France', 'Deal Size' : '30000', 'Description' : 'Pain Point: Persistent phishing attacks and email compromises leading to data leaks and compromised accounts. Description: The customer is struggling with the recurring threat of phishing attacks that infiltrate their systems, compromising sensitive data and risking the confidentiality of critical information.' }
                        , embedding_llm_output , sentiment))

# record #4

next_step = f"""
Action Items:
From Caroline, user community engaged to respond to questions. @Dataiku - How can we get initial data from user community/pull together PoV for client? Action (Asan/Ken (sp?)): In-person outreach to Deloitte users and follow-up to 5 responses received.
"""

cadence = f"""
null;06/20/2023: Contact Federico Gandolfo,federico.hernan.gandolfo@abc.com,+54.911.3204.4871 to discuss Deal support;07/05/2023: Contact Federico Gandolfo,federico.hernan.gandolfo@abc.com,+54.911.3204.4871 to discuss Deal support
"""

next_step_and_cadence = next_step + cadence

sentiment = indentify_sentiment(next_step_and_cadence)
llm_output = extract_llm_information(next_step_and_cadence)
print(f"{llm_output}/n")
embedding_llm_output = openai.embeddings.create(input= llm_output, model=model_id).data[0].embedding
session.execute(query, ('CUS100' , 'AWS', 'WS-8a7128a3', 'Teradyne, Inc.', llm_output,  {'Customer State' : 'Ile-de-France', 'Customer Country' : 'France', 'Deal Size' : '30000', 'Description' : 'Pain Point: Persistent phishing attacks and email compromises leading to data leaks and compromised accounts. Description: The customer is struggling with the recurring threat of phishing attacks that infiltrate their systems, compromising sensitive data and risking the confidentiality of critical information.' }
                        , embedding_llm_output , sentiment))

# record #4

next_step = f"""
Propsal did not go thru. No budget Left. Negative.
"""

cadence = f"""
No further follow up required.
"""

next_step_and_cadence = next_step + cadence

sentiment = indentify_sentiment(next_step_and_cadence)
llm_output = extract_llm_information(next_step_and_cadence)
print(f"{llm_output}/n")
embedding_llm_output = openai.embeddings.create(input= llm_output, model=model_id).data[0].embedding
session.execute(query, ('CUS100' , 'AWS', 'WS-8a7128a4', 'Teradyne, Inc.', llm_output,  {'Customer State' : 'Ile-de-France', 'Customer Country' : 'France', 'Deal Size' : '30000', 'Description' : 'Pain Point: Persistent phishing attacks and email compromises leading to data leaks and compromised accounts. Description: The customer is struggling with the recurring threat of phishing attacks that infiltrate their systems, compromising sensitive data and risking the confidentiality of critical information.' }
                        , embedding_llm_output , sentiment))



[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


Sentiment: positive
Challenges:
1. Deprioritization confirmed by Michael may affect project timeline and allocation of resources. 

Next Steps:
1. On 08/16/2023, review partner information updates and update opportunity details.
2. On 08/17, LR to connect with Partner to offer co-sell support. 

Open Items:
1. Meeting to be scheduled by Anjaney with Nirav/Amy on R&D.

Schedule a Meeting:
1. Anjaney to schedule a call with Nirav/Amy on R&D. Discussion of account executive interest./n
Sentiment: neutral


[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


Challenges:
1. Timely delivery of the recording and inputs from the demo held on 8/28.
2. Review of the Technical and Economic Proposal might be challenging if there's a lack of expertise in the team.

Next Steps:
1. Autumn needs to send the recording of the last call and the discussed inputs from the demo conducted on 8/28.
2. Ramesh is required to provide the recording to Caroline by early next week.
3. The team needs to review the Technical and Economic Proposal.

Open Items:
1. Completion of the task that has been assigned to Autumn.
2. Review of the Technical and Economic Proposal.

Schedule a Meeting:
No mention of a meeting to be scheduled./n


[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


Sentiment: positive
Challenges:
1. Coordinating with Federico Gandolfo for the deal support discussion.

Next Steps:
1. Execution of the joint sync set for 9/7.
2. Commence an enablement session followed by an in-person account mapping.
3. Caroline and Michael need to start coordinating.
4. Ensure EAI presence.

Open Items:
1. Deal support discussion with Federico Gandolfo.

Schedule a Meeting:
1. Joint sync set for 9/7.
2. A meeting with Federico Gandolfo on 07/05/2023 for deal support discussion./n


[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


Sentiment: positive
Challenges:
1. Initial data retrieval from the user community.
2. Pulling together Point of View (PoV) for the client.
3. Conducting in-person outreach to Deloitte users.
4. Follow-up on received responses.

Next Steps:
1. Contact Federico Gandolfo on 06/20/2023 and 07/05/2023 to discuss deal support.
2. Perform in-person outreach to Deloitte users. 

Open Items:
1. Response from the user community to Caroline's engagement.
2. Clarification on pulling together PoV for the client.

Schedule a Meeting:
1. Meeting with Federico Gandolfo on 06/20/2023 and 07/05/2023./n


[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


Sentiment: negative
Challenges:
1. Proposal did not go through.
2. There's no budget left.

Next Steps:
No further follow up required.

Open Items:
None

Schedule a Meeting:
Not applicable/n


# Opportunity Specific Queries

What is the customer sentiment on this opportunity?

In [22]:
cqlSelect = f'''SELECT * FROM workspan.customer_opportunities WHERE customer_id = 'CUS100' and partner_id = 'AWS' and opportunity_id = 'WS-7202838a'  ;'''
rows = session.execute(cqlSelect)
for row_i, row in enumerate(rows):
    print(f'\nRow {row_i}:')
    print(f'    customer_id:      {row.customer_id}')
    print(f'    partner_id:      {row.partner_id}')
    print(f'    opportunity_id:      {row.opportunity_id}')
    print(f'    customer_name:      {row.customer_name}')
    print(f'    sentiment:      {row.sentiment}')

print('\n...')


Row 0:
    customer_id:      CUS100
    partner_id:      AWS
    opportunity_id:      WS-7202838a
    customer_name:      Teradyne, Inc.
    sentiment:      positive

...


What are the next steps for this opportunity? Result can be further parsed to display only the next steps. Same query returns open items and challenges as well.  

In [23]:
cqlSelect = f'''SELECT * FROM workspan.customer_opportunities WHERE customer_id = 'CUS100' and partner_id = 'AWS' and opportunity_id = 'WS-7202838a';  '''
rows = session.execute(cqlSelect)
for row_i, row in enumerate(rows):
    print(f'\nRow {row_i}:')
    print(f'    customer_id:      {row.customer_id}')
    print(f'    partner_id:      {row.partner_id}')
    print(f'    opportunity_id:      {row.opportunity_id}')
    print(f'    customer_name:      {row.customer_name}')
    print(f'    llm_output:    \n  {row.llm_output}')

print('\n...')


Row 0:
    customer_id:      CUS100
    partner_id:      AWS
    opportunity_id:      WS-7202838a
    customer_name:      Teradyne, Inc.
    llm_output:    
  Challenges:
1. The deprioritized item from Michael.
   
Next Steps:
1. On 08/16/2023, you need to review partner information updates and update opportunity details.
2. On 8/17, LR needs to connect with the Partner to offer co-sell support.

Open Items:
1. Anjaney needs to schedule a meeting with Nirav/Amy on R&D.

Schedule a Meeting:
1. Anjaney is to schedule a call with Nirav/Amy on R&D.

...


# Customer Specific Queries (across multiple opportunities)

Identify the wins

In [24]:
cqlSelect = f'''SELECT * FROM workspan.customer_opportunities WHERE customer_id = 'CUS100' and partner_id = 'AWS' and sentiment = 'positive'  ;'''
rows = session.execute(cqlSelect)
for row_i, row in enumerate(rows):
    print(f'\nRow {row_i}:')
    print(f'    customer_id:      {row.customer_id}')
    print(f'    partner_id:      {row.partner_id}')
    print(f'    opportunity_id:      {row.opportunity_id}')
    print(f'    customer_name:      {row.customer_name}')
    print(f'    sentiment:      {row.sentiment}')

print('\n...')


Row 0:
    customer_id:      CUS100
    partner_id:      AWS
    opportunity_id:      WS-8a7128a3
    customer_name:      Teradyne, Inc.
    sentiment:      positive

Row 1:
    customer_id:      CUS100
    partner_id:      AWS
    opportunity_id:      WS-8a3b0348
    customer_name:      Teradyne, Inc.
    sentiment:      positive

Row 2:
    customer_id:      CUS100
    partner_id:      AWS
    opportunity_id:      WS-7202838a
    customer_name:      Teradyne, Inc.
    sentiment:      positive

...


Identify opportunities with next step to schedule a meeting

In [26]:
vectorsearchon = 'next action to set up a meeting'
embedding = openai.embeddings.create(input= vectorsearchon, model=model_id).data[0].embedding

cqlSelect = f'''SELECT * FROM workspan.customer_opportunities WHERE customer_id = 'CUS100' and partner_id = 'AWS' and llm_output : 'schedule a meeting' ORDER BY llm_output_embedding ANN OF {embedding} LIMIT 10;  '''
rows = session.execute(cqlSelect)
for row_i, row in enumerate(rows):
    print(f'\nRow {row_i}:')
    print(f'    customer_id:      {row.customer_id}')
    print(f'    partner_id:      {row.partner_id}')
    print(f'    opportunity_id:      {row.opportunity_id}')
    print(f'    customer_name:      {row.customer_name}')
    print(f'    sentiment:      {row.sentiment}')
    print(f'    llm_output:    \n  {row.llm_output}')

print('\n...')



Row 0:
    customer_id:      CUS100
    partner_id:      AWS
    opportunity_id:      WS-8a038b8a
    customer_name:      Teradyne, Inc.
    sentiment:      neutral
    llm_output:    
  Challenges:
1. Timely delivery of the recording and inputs from the demo held on 8/28.
2. Review of the Technical and Economic Proposal might be challenging if there's a lack of expertise in the team.

Next Steps:
1. Autumn needs to send the recording of the last call and the discussed inputs from the demo conducted on 8/28.
2. Ramesh is required to provide the recording to Caroline by early next week.
3. The team needs to review the Technical and Economic Proposal.

Open Items:
1. Completion of the task that has been assigned to Autumn.
2. Review of the Technical and Economic Proposal.

Schedule a Meeting:
No mention of a meeting to be scheduled.

Row 1:
    customer_id:      CUS100
    partner_id:      AWS
    opportunity_id:      WS-7202838a
    customer_name:      Teradyne, Inc.
    sentiment:    

I want to know more about the customer and the challenges so that it is possible to plan accordingly (Implement query using agent framework such as LangChain, LlamaIndex..)

In [27]:
vectorsearchon = 'find opportunity with listed challenges'
embedding = openai.embeddings.create(input= vectorsearchon, model=model_id).data[0].embedding

cqlSelect = f'''SELECT llm_output FROM workspan.customer_opportunities WHERE customer_id = 'CUS100' and partner_id = 'AWS' ORDER BY llm_output_embedding ANN OF {embedding} LIMIT 10;  '''
rows = session.execute(cqlSelect)
print(rows)
documents = []
for item in rows:
    documents.append(Document(text=str(item)))
    print(str(item))

index = ListIndex.from_documents(documents)

# set Logging to DEBUG for more detailed outputs
query_engine = index.as_query_engine()
response = query_engine.query("What are the Challenges?")

# visualize in console or web
print(response)
display(Markdown(f"<b>{response}</b>"))

Row(llm_output='Challenges:\n1. The deprioritized item from Michael.\n   \nNext Steps:\n1. On 08/16/2023, you need to review partner information updates and update opportunity details.\n2. On 8/17, LR needs to connect with the Partner to offer co-sell support.\n\nOpen Items:\n1. Anjaney needs to schedule a meeting with Nirav/Amy on R&D.\n\nSchedule a Meeting:\n1. Anjaney is to schedule a call with Nirav/Amy on R&D.')
Row(llm_output="Challenges:\n1. Proposal did not go through.\n2. There's no budget left.\n\nNext Steps:\nNo further follow up required.\n\nOpen Items:\nNone\n\nSchedule a Meeting:\nNot applicable")
Row(llm_output="Challenges:\n1. Initial data retrieval from the user community.\n2. Pulling together Point of View (PoV) for the client.\n3. Conducting in-person outreach to Deloitte users.\n4. Follow-up on received responses.\n\nNext Steps:\n1. Contact Federico Gandolfo on 06/20/2023 and 07/05/2023 to discuss deal support.\n2. Perform in-person outreach to Deloitte users. \n\nO

[nltk_data] Downloading package punkt to /tmp/llama_index...
[nltk_data]   Unzipping tokenizers/punkt.zip.


The challenges mentioned in the context information are as follows:
1. The deprioritized item from Michael.
2. Proposal did not go through.
3. There's no budget left.
4. Initial data retrieval from the user community.
5. Pulling together Point of View (PoV) for the client.
6. Conducting in-person outreach to Deloitte users.
7. Follow-up on received responses.
8. Timely delivery of the recording and inputs from the demo held on 8/28.
9. Review of the Technical and Economic Proposal might be challenging if there's a lack of expertise in the team.
10. Coordinating with Federico Gandolfo for the deal support discussion.
11.


<b>The challenges mentioned in the context information are as follows:
1. The deprioritized item from Michael.
2. Proposal did not go through.
3. There's no budget left.
4. Initial data retrieval from the user community.
5. Pulling together Point of View (PoV) for the client.
6. Conducting in-person outreach to Deloitte users.
7. Follow-up on received responses.
8. Timely delivery of the recording and inputs from the demo held on 8/28.
9. Review of the Technical and Economic Proposal might be challenging if there's a lack of expertise in the team.
10. Coordinating with Federico Gandolfo for the deal support discussion.
11.</b>